In [3]:
import numpy as np

import tempfile
import os

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

import matplotlib.pyplot as plt

In [4]:
# Load the dataset

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

2024-09-10 09:44:13.645288: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-09-10 09:44:13.647334: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-09-10 09:44:14.711794: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-10 09:44:14.711847: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 Ti with Max-Q Design computeCapability: 7.5
coreClock: 1.2GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 149.04GiB/s
2024-09-10 09:44:14.711869: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-09-10 09:44:14.786876: I tensorflow/str

In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

def reshape_img(image, label):
  return tf.reshape(image, (224, 224, 1)), label

def resize_img(image, label):
  return tf.image.resize(image, (224, 224)), label

ds_train = ds_train.map(
    resize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(
  reshape_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [6]:
ds_test = ds_test.map(
    resize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(
    reshape_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [7]:
model = keras.applications.MobileNetV2(
    input_shape=(224, 224, 1),
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


2024-09-10 09:44:39.892966: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-09-10 09:44:39.940510: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2592005000 Hz
2024-09-10 09:44:41.272583: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-09-10 09:44:51.233652: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 29261 of 60000
2024-09-10 09:45:02.230672: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 34681 of 60000


: 